In [1]:
from mio import *
#set_mio_log_level('debug')


# Introduction
Mio is a library designed to store and share protected information. It allows information to be stored in public and cloud locations (e.g., S3) while ensuring that only intended peers have access, and not the storage service providers.

### Identity
Each peer in the information sharing is identified by a cryptographic key, which allows for identification via signature and asymmetric encryption of data. An identity also includes a nickname, a human-readable name for easier management.

An identity consists of a public ID, which is a combination of the nickname and the public part of the cryptographic key.

In [2]:
i = Identity('Admin')  # New identity with name

print('ID: ', i.id)
print('Nick: ', i.nick)
print('Private key', i.private)

ID:  Admin.Aqo3jMBF5XCDLoPxAZ4zNvBOoflnpJr53TEfZRi0Nl5_8mfkWjdDJOGsY6U4Ql1bOceu3LUbGB2hKm3ObGWTkyI!
Nick:  Admin
Private key EUV9CKzIBT+7lrJho7u4NesjvYTeo5D0W4CwgJBcm2Ro+ABXgww4DR6dg+b0LWklNy3lRCrMBM6fJLPgugtFBPJn5Fo3QyThrGOlOEJdWznHrty1GxgdoSptzmxlk5Mi


### Local DB
The library requires a local SQLite database to manage metadata and encryption keys. By default, this database is named _mio.db_ and is created in the user's configuration folder.

In [3]:
DB.default()

/home/ea/.config/mio.db

### Safe
The information container is called a safe. The safe integrates a local database, the identity, and a storage URL to create a protected information storage system.

As of this writing, the library supports the following storage media: local filesystem, SFTP, S3, and WebDav. The table below shows the URL format for each storage type. The URL path includes the public ID of the safe's creator and a user-friendly name for the safe.

|Media|Format|Sample|
|-|-|-|
|local|file://_path_/_id_/_name_|file:///tmp/mio/Admin.A3Z1CR0wYMK_gXsRLkpowC3dVFC5rUNEeakiWPyb3D5l5VD1SXEFTxzQKEIvzNvKGEZGYp4yETo77SN+ViGP_00!/sample|
|S3|s3://_server_/_bucket_/_id_/_name_?a=_access_&s=_secret_|s3://d551285d92ed8fa4048fc09ca9113568.r2.cloudflarestorage.com/mio/Admin.A3Z1CR0wYMK_gXsRLkpowC3dVFC5rUNEeakiWPyb3D5l5VD1SXEFTxzQKEIvzNvKGEZGYp4yETo77SN+ViGP_00!/sample?a=acc5aba2f85d63536bbb45f085bb2b23&s=bcc49533aaaa46d929282b542ce598a12f7a4522dac8d5e9d403b39629484c2b&region=auto|


A new safe is created using the _create_ method.

In [4]:
url = 'file:///tmp/mio/{}/sample'.format(i)
s = Safe.create(DB.default(), i, url)
s

file:///tmp/mio/Admin.Aqo3jMBF5XCDLoPxAZ4zNvBOoflnpJr53TEfZRi0Nl5_8mfkWjdDJOGsY6U4Ql1bOceu3LUbGB2hKm3ObGWTkyI!/sample

At creation, only the creator belongs to the safe in both the _adm_ and _usr_ groups. Similar to Unix operating systems, a safe has multiple groups that define the permissions for different users (identities). The _get_groups_ method displays the available groups and their associated users.

In [5]:
s.get_groups()

{'adm': {'Admin.Aqo3jMBF5XCDLoPxAZ4zNvBOoflnpJr53TEfZRi0Nl5_8mfkWjdDJOGsY6U4Ql1bOceu3LUbGB2hKm3ObGWTkyI!': True},
 'usr': {'Admin.Aqo3jMBF5XCDLoPxAZ4zNvBOoflnpJr53TEfZRi0Nl5_8mfkWjdDJOGsY6U4Ql1bOceu3LUbGB2hKm3ObGWTkyI!': True}}

A user can be added to the safe using the _update_group_ method

In [6]:
alice = Identity('Alice')
s.update_group('usr', Safe.grant, [alice.id])

{'adm': {'Admin.Aqo3jMBF5XCDLoPxAZ4zNvBOoflnpJr53TEfZRi0Nl5_8mfkWjdDJOGsY6U4Ql1bOceu3LUbGB2hKm3ObGWTkyI!': True},
 'usr': {'Admin.Aqo3jMBF5XCDLoPxAZ4zNvBOoflnpJr53TEfZRi0Nl5_8mfkWjdDJOGsY6U4Ql1bOceu3LUbGB2hKm3ObGWTkyI!': True,
  'Alice.A8vvgvP1t4XETzl8vSn9Qhn+SnzHKuoxlgtT0RE8krqllJLVelcU8rJoMQ5jFro_ELYqya_MsdJ6xWYDkJIjcvY!': True}}

### Filesystem
A filesystem provides a file-oriented interface on top of a safe. Use the _put_data_ method to write a new file and the _list_ method to display the updated directory contents.

In [7]:
fs = s.fs()
fs.put_data('sub/test.txt', b'test')
fs.list()

[{'ID': 0,
  'Dir': '',
  'Name': 'sub',
  'IsDir': True,
  'GroupName': '',
  'Creator': '',
  'Size': 0,
  'ModTime': '1970-01-01T01:00:00+01:00',
  'Tags': [''],
  'Attributes': None,
  'LocalCopy': '',
  'CopyTime': '1970-01-01T01:00:00+01:00',
  'EncryptionKey': ''}]

In [8]:
fs.list('sub')

[{'ID': 195414434012143616,
  'Dir': 'sub',
  'Name': 'test.txt',
  'IsDir': False,
  'GroupName': 'usr',
  'Creator': 'Admin.Aqo3jMBF5XCDLoPxAZ4zNvBOoflnpJr53TEfZRi0Nl5_8mfkWjdDJOGsY6U4Ql1bOceu3LUbGB2hKm3ObGWTkyI!',
  'Size': 4,
  'ModTime': '2024-06-23T07:47:11.690123077+02:00',
  'Tags': [''],
  'Attributes': None,
  'LocalCopy': '',
  'CopyTime': '2024-06-23T07:47:11.691040302+02:00',
  'EncryptionKey': 'MA1qulHSEI5a7DyqGVZzHj2e3+tp2C3sOz3I9W0WrVO39rEv47ZCMMBRPc2xytHA'}]

In [9]:
fs.get_data('sub/test.txt')

b'test'

In [10]:
fs.delete('sub/test.txt')
fs.list()

[]

### Distributed Database
A SQL sqlite API where changes are propagated to other peer through the safe.

A new instance requires a group that will be used during the transfer of data through the safe. Only users that belong to the group will receive updates. 
During the creation of the instance same initialization  _ddl_  can be provided in input; each initialization SQL mut be preceed by a comment _-- INIT_


In [11]:
ddl = """
-- INIT
CREATE TABLE IF NOT EXISTS animal (name TEXT)
"""

ddls = {1.0: ddl}
d = s.database('usr', ddls)

The below code insert a row in the table and then retrieve all the values

In [20]:
d.exec('INSERT INTO animal VALUES (:name)', name = 'cat')
rows = d.query("SELECT * FROM animal ")
for row in rows:
    print(row)

['cat']


The methods _exec_ and _select_ operate locally. Only when the method _sync_ is called, changes propagates to other peers and at the same outgoing changes are applied.

In [13]:
d.sync()

The API supports a mechanism similar to placeholders defined in the initialization SQL. For instance

In [17]:
ddl = """
-- INIT
CREATE TABLE IF NOT EXISTS animal (name TEXT)

-- INSERT_ANIMAL
INSERT INTO animal VALUES (:name)

-- SELECT_ANIMALS
SELECT * FROM animal
"""

ddls = {1.0: ddl}
d = s.database('usr', ddls)

d.exec('INSERT_ANIMAL', name = 'cat')
rows = d.query('SELECT_ANIMALS')
for row in rows:
    print(row)


['cat']
['cat']
['cat']
['cat']
['cat']
['cat']
['cat']
['cat']
['cat']
['cat']
['cat']
['cat']
['cat']
['cat']
['cat']
['cat']
['cat']
['cat']
['cat']
['cat']
['cat']
['cat']
['cat']
['cat']
['cat']
['cat']
['cat']
['cat']
['cat']
['cat']
['cat']
['cat']
['cat']
['cat']


In [18]:
d.exec('DELETE FROM animal')

34

In [19]:
d.sync()